<a href="https://colab.research.google.com/github/aworklab/everydayPython/blob/main/ex_20220529.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 오늘의 목표
* RFM

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, time, date, timezone, timedelta
import os

# RFM

In [3]:
os.listdir()

['.config',
 'retail_data_transactions.csv',
 'ecommerce-data.csv',
 'retail_data_response.csv',
 'sample_data']

In [26]:
df = pd.read_csv('ecommerce-data.csv', encoding = 'unicode_escape')
df.columns = [a.lower() for a in df.columns]
df.invoicedate = pd.to_datetime(df.invoicedate)
df.head()

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [27]:
df = df[(df['country'] == 'United Kingdom') & (df['invoicedate'].dt.year == 2011)]
df.head()

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country
42481,539993,22386,JUMBO BAG PINK POLKADOT,10,2011-01-04 10:00:00,1.95,13313.0,United Kingdom
42482,539993,21499,BLUE POLKADOT WRAP,25,2011-01-04 10:00:00,0.42,13313.0,United Kingdom
42483,539993,21498,RED RETROSPOT WRAP,25,2011-01-04 10:00:00,0.42,13313.0,United Kingdom
42484,539993,22379,RECYCLING BAG RETROSPOT,5,2011-01-04 10:00:00,2.10,13313.0,United Kingdom
42485,539993,20718,RED RETROSPOT SHOPPER BAG,10,2011-01-04 10:00:00,1.25,13313.0,United Kingdom


In [28]:
df = df[~df.invoiceno.str.contains(r'^C')]
df.head()

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country
42481,539993,22386,JUMBO BAG PINK POLKADOT,10,2011-01-04 10:00:00,1.95,13313.0,United Kingdom
42482,539993,21499,BLUE POLKADOT WRAP,25,2011-01-04 10:00:00,0.42,13313.0,United Kingdom
42483,539993,21498,RED RETROSPOT WRAP,25,2011-01-04 10:00:00,0.42,13313.0,United Kingdom
42484,539993,22379,RECYCLING BAG RETROSPOT,5,2011-01-04 10:00:00,2.10,13313.0,United Kingdom
42485,539993,20718,RED RETROSPOT SHOPPER BAG,10,2011-01-04 10:00:00,1.25,13313.0,United Kingdom


In [29]:
df['price'] = df['quantity'] * df['unitprice']
df['date'] = df['invoicedate'].to_numpy().astype('datetime64[D]')

In [31]:
df = df[['customerid', 'date', 'price']].sort_values(['customerid', 'date'], ascending = [1, 0])
df.head()

,customerid,date,price
61619,12346.0,2011-01-18,77183.6
534566,12747.0,2011-12-07,243.0
534567,12747.0,2011-12-07,20.8
534568,12747.0,2011-12-07,39.6
534569,12747.0,2011-12-07,20.8


In [41]:
df = df[df.customerid.isin((df['customerid'].value_counts() > 1).index)]
df.head()

,customerid,date,price
61619,12346.0,2011-01-18,77183.6
534566,12747.0,2011-12-07,243.0
534567,12747.0,2011-12-07,20.8
534568,12747.0,2011-12-07,39.6
534569,12747.0,2011-12-07,20.8


In [61]:
info_r = df.groupby('customerid')['date'].max()
info_rd = df.groupby('customerid')['date'].apply(lambda x : 
                                       x.max() - datetime.strptime('2011-01-01', '%Y-%m-%d') + 
                                       np.timedelta64(1, 'D')).dt.days
info_f = df.groupby('customerid')['price'].size()
info_m = df.groupby('customerid')['price'].sum()

info = pd.concat([info_r, info_rd, info_f, info_m], axis = 1).reset_index()
info.columns = ['customerid', 'redency', 'redency_days', 'freq', 'money']
info.head()

,customerid,redency,redency_days,freq,money
0,12346.0,2011-01-18,18,1,77183.60
1,12747.0,2011-12-07,341,88,3489.74
2,12748.0,2011-12-09,343,3928,29491.60
3,12749.0,2011-12-06,340,199,4090.88
4,12820.0,2011-12-06,340,59,942.34
